In [ ]:
%reset 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from matplotlib import cm
import matplotlib.patches as patches
import pylab
from scipy.optimize import fsolve


DataPath_1='./Data/Suppl_Data_FS8_Stoch_BBBm_kIS_kRS_1'
DataPath_2='./Data/Data_F3_Stoch_BBBm'
DataPath_3='./Data/Suppl_Data_FS8_Stoch_BBBm_kIS_kRS_5'


%run ./DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
%run ./DataGenerationScripts/set_size.ipynb # loading function for pdf output sizer
%run ./DataGenerationScripts/aux_state_space.ipynb # loading auxiliarry functions for state space vizualization
%run ./DataGenerationScripts/load_animal_model_data.ipynb # loading function for pdf output sizer
%run ./DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions

In [ ]:
##### This cell checks whether data exists. If not, it is being generated #####
if not os.path.isdir(DataPath_1):
    os.makedirs(DataPath_1)
if not os.path.isfile(DataPath_1+'/Sim_1.npy'):
    print('Data not found. Running Data Generation script')
    RunningFromFigureScript=True
    %run ./DataGenerationScripts/Suppl_DataGeneration_FS8_Stoch_BBBm_kIS_kRS_1.ipynb
if not os.path.isdir(DataPath_2):
    os.makedirs(DataPath_2)
if not os.path.isfile(DataPath_2+'/Sim_1.npy'):
    print('Data not found. Running Data Generation script')
    RunningFromFigureScript=True
    %run ./DataGenerationScripts/DataGeneration_F3_Stoch_BBBm.ipynb
if not os.path.isdir(DataPath_3):
    os.makedirs(DataPath_3)
if not os.path.isfile(DataPath_3+'/Sim_1.npy'):
    print('Data not found. Running Data Generation script')
    RunningFromFigureScript=True
    %run ./DataGenerationScripts/Suppl_DataGeneration_FS8_Stoch_BBBm_kIS_kRS_5.ipynb

In [ ]:
font = {'size'   : 8}
# plt.rc('font', family='sans-serif') 
# plt.rc('font', serif='Helvetica Neue')
plt.rc('font',**{'family':'sans-serif','serif':['Helvetica Neue']})
plt.rc('font', **font)
plt.rc('text', usetex=True)

viridis = cm.get_cmap('viridis', 12)
color_scheme_5examples = [viridis(0.0)[0:3],viridis(0.3)[0:3],viridis(0.55)[0:3],viridis(0.75)[0:3],viridis(0.95)[0:3]]
alpha_value=0.45
marker_sizer=2.5
dt_vis=30


h=2.8*0.393701 #[cm] to  modification [inch] Height of output figure
w=2.8*0.393701 #[cm] to  modification [inch] Width of output figure
margin=0.4 # [1] margin ratio of output figure layout
size_scaler=2.5

# FIG S8A

In [ ]:
D_vis_vect=[0]

In [ ]:
params=load_default_parameters('rate')

params['k_IS']=1
params['k_RS']=1

locals().update(params)

B_max = 1.5
R_max = 1.5
B_min = -1.5
R_min = -1.5

for ii in range(0,len(D_vis_vect)):
    
    D_const=D_vis_vect[ii]


    R,B = np.mgrid[R_min:R_max:5000j, B_min:B_max:5000j]

    FP=[];
    f = K_SB*(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)-1)/(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)+1)
    U = 1./tau_B*(-B+k_IB*k_BI*B+f)
    V = 1./tau_R*(-R+k_BR*B+k_DR*D_const) 

#     velocity = np.sqrt(U*U + V*V)                
    FP=fixedpointsfinder(fun_dbdt,-1,1.5,100000) # Here we will check for steady states on given interval with given discritization step

    
    pylab.ion()
    fig0= plt.figure()
    ax0 = fig0.add_subplot(1, 1, 1)
    strm = ax0.streamplot(B, R, U, V, linewidth=1/marker_sizer, color=[0.75,0.75,0.75], density=[0.5,1.5])

    for cc in range(len(FP)):
        if FP[cc][0]>=Theta:
            col='grey'
        else:
            col='black' 

        mrksize = 10
        if FP[cc][1]=='Unstable':
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'o',color='white', markersize=mrksize/marker_sizer-3/marker_sizer)
        if FP[cc][1]=='Semistable':
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'o',color='white', markersize=mrksize/marker_sizer-3/marker_sizer)
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'x',color=col, markersize=mrksize/marker_sizer-3/marker_sizer)
        if FP[cc][1]=='Stable':
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
    
    ax0.contour(B,R,U,levels=[0.],colors='gray',linewidths=1.0)
    ax0.contour(B,R,V,levels=[0.],colors='gray',linewidths=1.0)
    
    threshold_position = Theta/k_BI
    plt.plot([threshold_position, threshold_position],[R_min,R_max],'--',color='tab:red',linewidth = 3.0/marker_sizer)
    

    plt.xlabel("Extent of blood-brain \n" 
               r"barrier disruption $B \approx I$")
    plt.ylabel("Degree of circuit \n"
               r"remodeling $R$")
    plt.title('$D$='+str(np.round(D_const*100)/100),fontsize=font['size'])

params=load_default_parameters('rate')
    
locals().update(params)

for ii in range(0,len(D_vis_vect)):
    
    D_const=D_vis_vect[ii]

    f = K_SB*(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)-1)/(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)+1)
    U = 1./tau_B*(-B+k_IB*k_BI*B+f)
    V = 1./tau_R*(-R+k_BR*B+k_DR*D_const) 

    strm = ax0.streamplot(B, R, U, V, linewidth=1/marker_sizer, color=[0.75,0.75,0.75], density=[0.,0.])
    
    ax0.contour(B,R,U,levels=[0.],colors='black',linewidths=1.0,linestyles='dashed')
    ax0.contour(B,R,V,levels=[0.],colors='black',linewidths=1.0,linestyles='dashed')
    
ax0.set_xlim([B_min,B_max])
ax0.set_ylim([R_min,R_max])

plt.gcf().subplots_adjust(bottom=margin)
plt.gcf().subplots_adjust(left=margin)
set_size(1.0*w,1.0*w)

fig0.savefig('./Figures/Supplementary/Fig_S8A.pdf')  

# FIG S8B

In [ ]:
params=load_default_parameters('rate')

params['k_IS']=5
params['k_RS']=5

locals().update(params)

B_max = 1.5
R_max = 1.5
B_min = -1.5
R_min = -1.5

for ii in range(0,len(D_vis_vect)):
    
    D_const=D_vis_vect[ii]


    R,B = np.mgrid[R_min:R_max:5000j, B_min:B_max:5000j]

    FP=[];
    f = K_SB*(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)-1)/(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)+1)
    U = 1./tau_B*(-B+k_IB*k_BI*B+f)
    V = 1./tau_R*(-R+k_BR*B+k_DR*D_const) 

#     velocity = np.sqrt(U*U + V*V)                
    FP=fixedpointsfinder(fun_dbdt,-1,1.5,100000) # Here we will check for steady states on given interval with given discritization step

    
    pylab.ion()
    fig0= plt.figure()
    ax0 = fig0.add_subplot(1, 1, 1)
    strm = ax0.streamplot(B, R, U, V, linewidth=1/marker_sizer, color=[0.75,0.75,0.75], density=[0.5,1.5])

    for cc in range(len(FP)):
        if FP[cc][0]>=Theta:
            col='grey'
        else:
            col='black' 

        mrksize = 10
        if FP[cc][1]=='Unstable':
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'o',color='white', markersize=mrksize/marker_sizer-3/marker_sizer)
        if FP[cc][1]=='Semistable':
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'o',color='white', markersize=mrksize/marker_sizer-3/marker_sizer)
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'x',color=col, markersize=mrksize/marker_sizer-3/marker_sizer)
        if FP[cc][1]=='Stable':
            plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
    
    ax0.contour(B,R,U,levels=[0.],colors='gray',linewidths=1.0)
    ax0.contour(B,R,V,levels=[0.],colors='gray',linewidths=1.0)
    
    threshold_position = Theta/k_BI
    plt.plot([threshold_position, threshold_position],[R_min,R_max],'--',color='tab:red',linewidth = 3.0/marker_sizer)
    

    plt.xlabel("Extent of blood-brain \n" 
               r"barrier disruption $B \approx I$")
    plt.ylabel("Degree of circuit \n"
               r"remodeling $R$")
    plt.title('$D$='+str(np.round(D_const*100)/100),fontsize=font['size'])

params=load_default_parameters('rate')
    
locals().update(params)

for ii in range(0,len(D_vis_vect)):
    
    D_const=D_vis_vect[ii]

    f = K_SB*(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)-1)/(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)+1)
    U = 1./tau_B*(-B+k_IB*k_BI*B+f)
    V = 1./tau_R*(-R+k_BR*B+k_DR*D_const) 

    strm = ax0.streamplot(B, R, U, V, linewidth=1/marker_sizer, color=[0.75,0.75,0.75], density=[0.,0.])
    
    ax0.contour(B,R,U,levels=[0.],colors='black',linewidths=1.0,linestyles='dashed')
    ax0.contour(B,R,V,levels=[0.],colors='black',linewidths=1.0,linestyles='dashed')
    
ax0.set_xlim([B_min,B_max])
ax0.set_ylim([R_min,R_max])

plt.gcf().subplots_adjust(bottom=margin)
plt.gcf().subplots_adjust(left=margin)
set_size(1.0*w,1.0*w)

fig0.savefig('./Figures/Supplementary/Fig_S8B.pdf')  

# FIG S8C

In [ ]:
##### Calculating latent period duration for 4 simulated intensities: matched input intensity; decreased via lowering concentration; decreased via shortening; increased via prolongation #####
N=30

day_first_seiz=np.zeros(N)
for ii in range(0,N): # Here we gather and extract data about seizures on respective days 
    listt=np.load(DataPath_1+'/Sim_'+str(ii+1)+'.npy',allow_pickle=True)
    S_vec=listt[1]['S_vec'].copy()
    t_vec=listt[1]['t_vec'].copy()
    seiz_days=np.ceil(t_vec[S_vec!=0])
    day_first_seiz[ii]=seiz_days[0]
day_first_seiz_sim=day_first_seiz
LP_mean_k1=np.mean(day_first_seiz)   
LP_sem_k1=np.std(day_first_seiz)/np.sqrt(N)

day_first_seiz=np.zeros(N)
for ii in range(0,N): # Here we gather and extract data about seizures on respective days 
    listt=np.load(DataPath_2+'/Sim_'+str(ii+1)+'.npy',allow_pickle=True)
    S_vec=listt[1]['S_vec'].copy()
    t_vec=listt[1]['t_vec'].copy()
    seiz_days=np.ceil(t_vec[S_vec!=0])
    day_first_seiz[ii]=seiz_days[0]
day_first_seiz_simdec=day_first_seiz
LP_mean_k2=np.mean(day_first_seiz)   
LP_sem_k2=np.std(day_first_seiz)/np.sqrt(N)

day_first_seiz=np.zeros(N)
for ii in range(0,N): # Here we gather and extract data about seizures on respective days 
    listt=np.load(DataPath_3+'/Sim_'+str(ii+1)+'.npy',allow_pickle=True)
    S_vec=listt[1]['S_vec'].copy()
    t_vec=listt[1]['t_vec'].copy()
    seiz_days=np.ceil(t_vec[S_vec!=0])
    day_first_seiz[ii]=seiz_days[0]
day_first_seiz_simshort=day_first_seiz
LP_mean_k5=np.mean(day_first_seiz)   
LP_sem_k5=np.std(day_first_seiz)/np.sqrt(N)

In [ ]:
data = load_animal_model_data('Weissberg_et_al_2015')

fig, ax = plt.subplots()
vec=np.array([1,3])
plt.bar(np.array(vec),np.array([data['LatentPeriod_mean'],LP_mean_k2]), color='black',
       yerr=[data['LatentPeriod_sem'], LP_sem_k2],
       align='center',
       alpha=1,
       ecolor='black',
       capsize=10/3*2, width=0.75/3*2)
vec=np.array([2,4])
plt.bar(np.array(vec),np.array([ LP_mean_k1, LP_mean_k5]), color='grey',
       yerr=[LP_sem_k1, LP_sem_k5],
       align='center',
       alpha=1,
       ecolor='grey',
       capsize=10/3*2, width=0.8/3*2)

vec=np.array([1,2,3,4])
plt.xticks(np.array(vec), ['(Weissberg \n et al., 2015),\n N = 10',
                             r"1",
                             r"2" "\n Original value",
                             r"5"], fontsize=7.5)

plt.yticks(np.arange(0,10))
plt.ylabel('Latent period duration, days')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.gcf().subplots_adjust(bottom=margin)
plt.gcf().subplots_adjust(left=margin)
set_size(3/5*4*w,w)
fig.savefig('./Figures/Supplementary/Fig_S8C.pdf')